In [ ]:
import ROOT as root
from ROOT import gROOT
import numpy as np
from math import pi
from os import mkdir

In [ ]:
#Import Delphes
delphesPath = '/home/santiago/MG5_aMC_v2_6_5/Delphes'
root.gSystem.AddDynamicPath(delphesPath)
#root.gROOT.ProcessLine("#include <math.h>")
root.gSystem.Load("libDelphes")

#Images path
# imgPath = "./images/cut1_MET_inf_300/"
imgPath = "./images/cut0_no_cut/"

In [ ]:
try:
    mkdir(imgPath)
except OSError:
    print ("Creation of the directory %s failed" % imgPath)
else:
    print ("Successfully created the directory %s " % imgPath)

In [ ]:
#Maximum number of JETS
nJets = 5

#Minimum MET value
minMET = 300

In [ ]:
def absDeltaEta(eta1, eta2):
    return abs(eta2-eta1)

def deltaPhi(phi1,phi2):
    PHI=phi1-phi2
    if PHI >= pi:
        PHI -= 2*pi
    elif PHI < -1*pi:
        PHI += 2*pi
    return PHI

def mass(jetBranch):
    """
    @brief   Returns the invariant mass of the first two jets
    
    @warning Only use if the event has two or more jets
    """
    jet1 = root.TLorentzVector()
    jet2 = root.TLorentzVector()

    pt1, pt2 = jetBranch.At(0).PT, jetBranch.At(1).PT
    eta1, eta2 = jetBranch.At(0).Eta, jetBranch.At(1).Eta
    phi1, phi2 = jetBranch.At(0).Phi, jetBranch.At(1).Phi
    m1, m2 = jetBranch.At(0).Mass, jetBranch.At(1).Mass

    jet1.SetPtEtaPhiE(pt1,eta1,phi1,m1)
    jet1.SetPtEtaPhiE(pt2,eta2,phi2,m2)

    jet12 = jet1 + jet2
    jetMass = jet12.M()
    return abs(jetMass)
    

def maxValueHist(h1, h2):
    """
    Returns the maximum value of two histograms
    """
    return max(h1.GetBinContent(h1.GetMaximumBin()),
               h2.GetBinContent(h2.GetMaximumBin()))

def plotHist(c,h1,h2=None):
    """
    @brief Plot two histograms in the same canvas
           nortmalized to one
    @param c  Canvas
    @param h1 Histogram 1
    @param h2 Histogram 2
    """
    c.Clear()
    if h2 is not None:
        h1.Scale(1/h1.Integral())
        h2.Scale(1/h2.Integral())
        h1.SetMaximum(maxValueHist(h1,h2)*1.1)    
        h1.Draw()
        h2.SetLineColor(3)
        h2.Draw("SAME")
        label = root.TLegend(0,0,200,0.014)    
        label.AddEntry(h1,"Background")
        label.AddEntry(h2,"Signal")
        label.Draw("SAME")        
    else:
        h1.SetMaximum(maxValueHist(h1,h1)*1.1)    
        h1.Draw()        
    c.SaveAs(imgPath+h1.GetName()+'.png')
        

In [ ]:
try:
  root.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
  root.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
  print ("Delphes classes imported")
except:
  pass

In [ ]:
treeName = 'Delphes'
backgroundPath = './data/background/'
# signalPath = './data/VBF_MDSimp_EWKExcluded/DMSimpSpin1_MY1000_MX100.root'
signalPath = './data/VBF_MDSimp_EWKExcluded/daniel_run_05.root'


#Declaration of TCanvas
c1 = root.TCanvas("c1")

# Background 
dataChain = root.TChain(treeName)
dataChain.Add(backgroundPath + "*.root")
treeReader = root.ExRootTreeReader(dataChain)
numOfEntries = treeReader.GetEntries()

# Signal
f = root.TFile(signalPath)
sigTree = f.Get(treeName)
sigTreeReader = root.ExRootTreeReader(sigTree)

In [ ]:
# Selection of the branches
bHTbranch = treeReader.UseBranch("ScalarHT")
bJetBranch = treeReader.UseBranch("Jet")
bMETBranch = treeReader.UseBranch("MissingET")

#Number of Bins
nBins = int(np.sqrt(numOfEntries)) +1

# Histograms
bPtHist = []
bEtaJetHist = []
bDeltaPhiHist = []
bDeltaEtaHist = []
for i in range(nJets):
    bPtHist.append(root.TH1F("Pt{0}Histo".format(i),"Pt_{0}".format(i),1000,0,2000))
    bEtaJetHist.append(root.TH1F("EtaJ{0}Histo".format(i),"Eta_[J_{0}]".format(i),100,-5,5))
    bDeltaPhiHist.append(root.TH1F("hb{0}DeltaPhi".format(i), "|\Delta\phi(j{0},MET)|".format(i), 100, 0, 10))
    bDeltaEtaHist.append(root.TH1F("hb{0}DeltaEta".format(i), "|\Delta\eta(j{0},MET)|".format(i), 100, 0, 10))
bHTHist = root.TH1F("bHTHist","Signal HT",200,0,1000)
bRatioPtHist = root.TH1F("bRatioPtHist","Pt_{J_{1}}/Pt_{J_{2}}",nBins,0,20)
bPhiMETHist = root.TH1F("bPhiMETHist","Phi_{MET}",100,-5,5)
hBMass = root.TH1F("hBMass", "M(j1,j2)", 150,0,1500)
hBMetET = root.TH1F("hbMetET", "MET_ET", 400,0.0, 2000)

# Selection of the branches
sHTBranch = sigTreeReader.UseBranch("ScalarHT")
sJetBranch = sigTreeReader.UseBranch("Jet")
sMETBranch = sigTreeReader.UseBranch("MissingET")

# Histograms
nBins = int(np.sqrt(sigTreeReader.GetEntries())) +1

sPtHist = []
sEtaJetHist = []
sDeltaPhiHist = []
sDeltaEtaHist = []
for i in range(nJets):
    sPtHist.append(root.TH1F("sigPt{0}Histo".format(i),"Pt_{0}".format(i),1000,0,2000))
    sEtaJetHist.append(root.TH1F("sigEtaJ{0}Histo".format(i),"Eta_[J_{0}]".format(i),100,-5,5))
    sDeltaPhiHist.append(root.TH1F("hs{0}DeltaPhi".format(i), "|\Delta\phi(j{0},MET)|".format(i), 100, 0, 10))
    sDeltaEtaHist.append(root.TH1F("hs{0}DeltaEta".format(i), "|\Delta\eta(j{0},MET)|".format(i), 100, 0, 10))
sHTHist = root.TH1F("bHTHist","HT",200,0,1000)
sRatioPtHist = root.TH1F("sRatioPtHist","Pt_{J_{1}}/Pt_{J_{2}}",nBins,0,20)
sPhiMETHist = root.TH1F("sPhiMETHist","Phi_{MET}",100,-5,5)
hSMass = root.TH1F("hSMass", "M(j1,j2)", 150,0,1500)
hSMetET = root.TH1F("hsMetET", "MET_ET", 400, 0.0, 2000)


In [ ]:
for entry in xrange(numOfEntries):
    treeReader.ReadEntry(entry)
    
    #cuts
    if (bJetBranch.GetEntries() < 2) or(bJetBranch.At(0).Eta*bJetBranch.At(1).Eta >= 0):
        continue
    if bMETBranch.At(0).MET < minMET: continue
    
    
    if bHTbranch.GetEntries() > 0 :
        bHTHist.Fill(bHTbranch.At(0).HT)
    
   
    n =  nJets if bJetBranch.GetEntries()>5 else bJetBranch.GetEntries()
    for i in range(n):
        bPtHist[i].Fill(bJetBranch.At(i).PT)
        bEtaJetHist[i].Fill(bJetBranch.At(i).Eta)
        bDeltaEtaHist[i].Fill(absDeltaEta(bMETBranch.At(0).Eta, bJetBranch.At(i).Eta))
        bDeltaPhiHist[i].Fill(deltaPhi(bMETBranch.At(0).Phi,bJetBranch.At(i).Phi))

    bRatioPtHist.Fill(bJetBranch.At(0).PT / bJetBranch.At(1).PT)
    hBMass.Fill(mass(bJetBranch))
    
    bPhiMETHist.Fill(bMETBranch.At(0).Phi)
    hBMetET.Fill(bMETBranch.At(0).MET)

In [ ]:
for entry in xrange(sigTreeReader.GetEntries()):
    sigTreeReader.ReadEntry(entry)
    
    #cuts
    if (sJetBranch.GetEntries() < 2) or (sJetBranch.At(0).Eta*sJetBranch.At(1).Eta >= 0):
        continue
    if sMETBranch.At(0).MET < minMET: continue
    
    if sHTBranch.GetEntries() > 0 :
        sHTHist.Fill(sHTBranch.At(0).HT)
        
    n =  nJets if sJetBranch.GetEntries()>5 else sJetBranch.GetEntries()
    for i in range(n):
        sPtHist[i].Fill(sJetBranch.At(i).PT)
        sEtaJetHist[i].Fill(sJetBranch.At(i).Eta)
        sDeltaEtaHist[i].Fill(absDeltaEta(sMETBranch.At(0).Eta, sJetBranch.At(i).Eta))
        sDeltaPhiHist[i].Fill(deltaPhi(sMETBranch.At(0).Phi, sJetBranch.At(i).Phi))

    sRatioPtHist.Fill(sJetBranch.At(0).PT / sJetBranch.At(1).PT)
    hSMass.Fill(mass(sJetBranch))
    
    sPhiMETHist.Fill(sMETBranch.At(0).Phi)
    hSMetET.Fill(sMETBranch.At(0).MET)

<H2>Gráficas</H2>
<H3>HT</H3>

In [ ]:
plotHist(c1,bHTHist,sHTHist)

<H3>PT</H3>

In [ ]:
#for i in range(2):
#    plotHist(c1,bPtHist[i],sPtHist[i])
plotHist(c1,bPtHist[0],sPtHist[0])

In [ ]:
plotHist(c1,bPtHist[1],sPtHist[1])

<H3>Eta para los Jets 0 y 1</H3>

In [ ]:
#for i in range(2):
#    plotHist(c1,bEtaJetHist[i],sEtaJetHist[i])
plotHist(c1,bEtaJetHist[0],sEtaJetHist[0])

In [ ]:
plotHist(c1,bEtaJetHist[1],sEtaJetHist[1])

<H3>PT(j1)/PT(j2)</H3>

In [ ]:
plotHist(c1,bRatioPtHist, sRatioPtHist)

<H3>Phi MET</H3>

In [ ]:
plotHist(c1,bPhiMETHist,sPhiMETHist)

<H3>Delta de Eta para los Jets</H3>

In [ ]:
#for i in range(nJets):
#    plotHist(c1,bDeltaEtaHist[i],sDeltaEtaHist[i])
plotHist(c1,bDeltaEtaHist[0],sDeltaEtaHist[0])

In [ ]:
plotHist(c1,bDeltaEtaHist[1],sDeltaEtaHist[1])

In [ ]:
plotHist(c1,bDeltaEtaHist[2],sDeltaEtaHist[2])

In [ ]:
plotHist(c1,bDeltaEtaHist[3],sDeltaEtaHist[3])

In [ ]:
plotHist(c1,bDeltaEtaHist[4],sDeltaEtaHist[4])

<H3>Delta de phi para los Jets</H3>

In [ ]:
#for i in range(nJets):
#    plotHist(c1,bDeltaPhiHist[i],sDeltaPhiHist[i])
plotHist(c1,bDeltaPhiHist[0],sDeltaPhiHist[0])

In [ ]:
plotHist(c1,bDeltaPhiHist[1],sDeltaPhiHist[1])

In [ ]:
plotHist(c1,bDeltaPhiHist[2],sDeltaPhiHist[2])

In [ ]:
plotHist(c1,bDeltaPhiHist[3],sDeltaPhiHist[3])

In [ ]:
plotHist(c1,bDeltaPhiHist[4],sDeltaPhiHist[4])

<H3>Masa invariante</H3>

In [ ]:
plotHist(c1,hBMass,hSMass)

<H3>MET</H3>

In [ ]:
plotHist(c1,hBMetET,hSMetET)

<H3>Cálculo de la significancia</H3>

In [ ]:
bH = hBMetET
sH = hSMetET
zHist = bH.Clone("z_{MET}")
# bH = hBMass
# sH = hSMass
# zHist = bH.Clone("z_{m(J_{0},J_{1})}")
# bH = bPtHist[1]
# sH = sPtHist[1]
# zHist = bH.Clone("z_{Pt_{1}}")


L= (0.001)**(-1)
sigmaB = 72.38
sigmaS = 13.76

bH.Scale((L*sigmaB / numOfEntries))
sH.Scale((L*sigmaS / sigTreeReader.GetEntries()))

for i in range(zHist.GetNbinsX()):
    s = sH.Integral(i+1,zHist.GetNbinsX())
    b = bH.Integral(i+1,zHist.GetNbinsX())
    if s+b != 0:        
        zHist.SetBinContent(i+1, s/np.sqrt(s+b))
    else:
        zHist.SetBinContent(i+1, 0)
    zHist.SetBinError(i+1,0)
    
plotHist(c1,zHist)